In [1]:
# Menginstall pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=07e10a2c182d6f6ffa9be5d569ba368089fecf4a06ab7c765aeee574d13fb873
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Mengimpor modul SparkSession dari PySpark.
from pyspark.sql import SparkSession

# Inisialisasi SparkSession dalam variabel spark
spark = SparkSession.builder.getOrCreate()


# SparkContext (sc) digunakan untuk mengelola komputasi Spark
sc = spark.sparkContext

In [3]:
# Membaca dataset people.csv
people_df = spark.read.csv('people.csv', header=True, inferSchema=True)

# Menampilkan tipe data people_df
print("The type of people_df is", type(people_df))

The type of people_df is <class 'pyspark.sql.dataframe.DataFrame'>


In [4]:
# Menampilkan 10 baris pertama pada dataset
people_df.show(10)

print("There are {} rows in the people_df DataFrame.".format(people_df.count()))

print("There are {} columns in the people_df DataFrame and their names are {}".format(len(people_df.columns), people_df.columns))

+---+---------+----------------+------+-------------------+
|_c0|person_id|            name|   sex|      date of birth|
+---+---------+----------------+------+-------------------+
|  0|      100|  Penelope Lewis|female|1990-08-31 00:00:00|
|  1|      101|   David Anthony|  male|1971-10-14 00:00:00|
|  2|      102|       Ida Shipp|female|1962-05-24 00:00:00|
|  3|      103|    Joanna Moore|female|2017-03-10 00:00:00|
|  4|      104|  Lisandra Ortiz|female|2020-08-05 00:00:00|
|  5|      105|   David Simmons|  male|1999-12-30 00:00:00|
|  6|      106|   Edward Hudson|  male|1983-05-09 00:00:00|
|  7|      107|    Albert Jones|  male|1990-09-13 00:00:00|
|  8|      108|Leonard Cavender|  male|1958-08-08 00:00:00|
|  9|      109|  Everett Vadala|  male|2005-05-24 00:00:00|
+---+---------+----------------+------+-------------------+
only showing top 10 rows

There are 100000 rows in the people_df DataFrame.
There are 5 columns in the people_df DataFrame and their names are ['_c0', 'person_i

In [5]:
people_df_sub = people_df.select('name', 'sex', 'date of birth')

people_df_sub.show(10)

+----------------+------+-------------------+
|            name|   sex|      date of birth|
+----------------+------+-------------------+
|  Penelope Lewis|female|1990-08-31 00:00:00|
|   David Anthony|  male|1971-10-14 00:00:00|
|       Ida Shipp|female|1962-05-24 00:00:00|
|    Joanna Moore|female|2017-03-10 00:00:00|
|  Lisandra Ortiz|female|2020-08-05 00:00:00|
|   David Simmons|  male|1999-12-30 00:00:00|
|   Edward Hudson|  male|1983-05-09 00:00:00|
|    Albert Jones|  male|1990-09-13 00:00:00|
|Leonard Cavender|  male|1958-08-08 00:00:00|
|  Everett Vadala|  male|2005-05-24 00:00:00|
+----------------+------+-------------------+
only showing top 10 rows



In [6]:
# QUIZ
# Berapakah jumlah data sebelum dan sesudah di remove duplicate?
# Bersihkan dataset jenis kelamin null dan usia <0 dan >100
# Hitung usia dari masing-masing orang berdasarkan tanggal hari ini
# Kelompokan usia semua orang menjadi 0-17, 18-25, 25-35, 35-45, 45-50, 50-55, 55-100
# Hitung jumlah masing-masing jenis kelamin untuk masing-masing kelompok usia
# Tuliskan source code dan hasilnya di classroom ya

people_df_sub_nodup = people_df_sub.dropDuplicates()
print("There were {} rows before removing duplicates, and {} rows after removing duplicates"
    .format(people_df_sub.count(), people_df_sub_nodup.count()))

There were 100000 rows before removing duplicates, and 99998 rows after removing duplicates


In [7]:
print(people_df.count())
print(people_df_sub_nodup.count())

100000
99998


In [8]:
# Jumlah data sebelum dan sesudah di remove duplicate?
print("Jumlah data sebelum menghapus duplikat:", people_df_sub.count())
people_df_sub_nodup = people_df_sub.dropDuplicates()
print("Jumlah data setelah menghapus duplikat:", people_df_sub_nodup.count())


Jumlah data sebelum menghapus duplikat: 100000
Jumlah data setelah menghapus duplikat: 99998


In [9]:
from pyspark.sql.functions import current_date, col,  count
from pyspark.sql.functions import datediff, when

# Membersihkan NULL pada kolom 'sex'
people_df_cleaned = people_df_sub_nodup.filter(people_df_sub_nodup['sex'].isNotNull())

# Menghitung usia berdasarkan tanggal hari ini
today_date = current_date()
people_df_cleaned = people_df_cleaned.withColumn('age', datediff(today_date, col('date of birth')) / 365)

# Membersihkan usia < 0 dan > 100
people_df_cleaned = people_df_cleaned.filter((people_df_cleaned['age'] > 0) & (people_df_cleaned['age'] <= 100))
people_df_cleaned.show()

+---------------+------+-------------------+------------------+
|           name|   sex|      date of birth|               age|
+---------------+------+-------------------+------------------+
| Meredith Valez|female|1986-12-13 00:00:00|36.863013698630134|
| Jeffery Cowley|  male|1973-06-22 00:00:00|50.347945205479455|
|Otis Richardson|  male|1964-08-15 00:00:00|59.205479452054796|
| Shirley Parado|female|1993-06-27 00:00:00| 30.32054794520548|
|Kimberly Madore|female|1982-04-27 00:00:00| 41.49589041095891|
| Barbara Warren|female|2015-12-08 00:00:00| 7.857534246575343|
|  Joseph Tucker|  male|1959-07-19 00:00:00| 64.28493150684932|
| Barbara Kearns|female|1996-02-26 00:00:00| 27.65205479452055|
| Jessica Santos|female|1990-03-06 00:00:00|33.632876712328766|
|    John Kerney|  male|1987-03-01 00:00:00| 36.64931506849315|
|   Stacy Mardis|female|1996-05-07 00:00:00|27.457534246575342|
|    George Taul|  male|1991-08-24 00:00:00| 32.16438356164384|
|  Mary Williams|female|1970-10-13 00:00

In [10]:
# Kelompokan usia semua orang menjadi 0-17, 18-25, 25-35, 35-45, 45-50, 50-55, 55-100
from pyspark.sql.functions import when

people_df_cleaned = people_df_cleaned.withColumn('age_group',
    when((col('age') >= 0) & (col('age') <= 17), '0-17')
    .when((col('age') >= 17) & (col('age') <= 25), '17-25')
    .when((col('age') >= 25) & (col('age') <= 35), '25-35')
    .when((col('age') >= 35) & (col('age') <= 45), '35-45')
    .when((col('age') >= 45) & (col('age') <= 50), '45-50')
    .when((col('age') >= 50) & (col('age') <= 55), '50-55')
    .when((col('age') >= 55) & (col('age') <= 100), '55-100')
    .otherwise('Unknown')
)
people_df_cleaned.show()

+---------------+------+-------------------+------------------+---------+
|           name|   sex|      date of birth|               age|age_group|
+---------------+------+-------------------+------------------+---------+
| Meredith Valez|female|1986-12-13 00:00:00|36.863013698630134|    35-45|
| Jeffery Cowley|  male|1973-06-22 00:00:00|50.347945205479455|    50-55|
|Otis Richardson|  male|1964-08-15 00:00:00|59.205479452054796|   55-100|
| Shirley Parado|female|1993-06-27 00:00:00| 30.32054794520548|    25-35|
|Kimberly Madore|female|1982-04-27 00:00:00| 41.49589041095891|    35-45|
| Barbara Warren|female|2015-12-08 00:00:00| 7.857534246575343|     0-17|
|  Joseph Tucker|  male|1959-07-19 00:00:00| 64.28493150684932|   55-100|
| Barbara Kearns|female|1996-02-26 00:00:00| 27.65205479452055|    25-35|
| Jessica Santos|female|1990-03-06 00:00:00|33.632876712328766|    25-35|
|    John Kerney|  male|1987-03-01 00:00:00| 36.64931506849315|    35-45|
|   Stacy Mardis|female|1996-05-07 00:

In [11]:
# Menampilkan jumlah masing-masing jenis kelamin untuk masing-masing kelompok usia

from pyspark.sql.functions import col, datediff, current_date, when, count

sorted_gender_age_group_counts = people_df_cleaned.groupBy("sex", "age_group").agg(count("*").alias("count"))
sorted_gender_age_group_counts = sorted_gender_age_group_counts.orderBy("sex", "age_group")
print("Jumlah data: ", people_df_cleaned.count())
sorted_gender_age_group_counts.show()


Jumlah data:  93980
+------+---------+-----+
|   sex|age_group|count|
+------+---------+-----+
|female|     0-17| 4677|
|female|    17-25| 4776|
|female|    25-35| 8137|
|female|    35-45| 9539|
|female|    45-50| 4547|
|female|    50-55| 3980|
|female|   55-100|11317|
|  male|     0-17| 4758|
|  male|    17-25| 4571|
|  male|    25-35| 8341|
|  male|    35-45| 9605|
|  male|    45-50| 4558|
|  male|    50-55| 3960|
|  male|   55-100|11214|
+------+---------+-----+

